In [ ]:
import torch
import torchaudio
import torchaudio.sox_effects as sox
import numpy as np
from IPython.display import Audio

In [ ]:
sox.init_sox_effects()

In [ ]:
sox.effect_names()

In [ ]:
#Effect params
from torchaudio.utils.sox_utils import list_effects
list_effects()

In [ ]:
waveform, sample_rate = torchaudio.load('/home/oriol_colome_font_epidemicsound_/Master-Thesis/datasets/SALAMI/audio/2/audio.mp3')
waveform = waveform.mean(dim=0, keepdim=True)  # convert stereo to mono
Audio(waveform, rate=sample_rate)

In [ ]:
def add_noise_with_snr(waveform, snr_range):
        # Generate white noise
        noise = torch.normal(0, 1, waveform.shape)

        # Calculate signal and noise power
        signal_power = torch.sum(waveform ** 2)
        noise_power = torch.sum(noise ** 2)

        # Calculate the scaling factor for the noise
        scale_factor = torch.sqrt(signal_power / (noise_power * 10**(snr_range / 10)))

        # Scale the noise
        scaled_noise = noise * scale_factor

        # Add noise to the signal
        noisy_waveform = waveform + scaled_noise

        return noisy_waveform

In [ ]:
# Define the effect parameters using numpy
gain = np.random.randint(-12, 0)
pitch = np.random.randint(-1200, 1200)
reverb_params = [np.random.randint(0, 100)] * 3
chorus_params = [
    round(np.random.uniform(0.1, 1.0), 1),
    round(np.random.uniform(0.1, 1.0), 1),
    np.random.randint(20, 55),
    round(np.random.uniform(0.1, 0.9), 1),
    round(np.random.uniform(0.1, 2.0), 2),
    np.random.randint(2, 5),
    np.random.choice(["-s", "-t"]),
]
drive = np.random.randint(0, 30)
stretch = round(np.random.uniform(0.9, 1.1), 1)
speed = np.random.uniform(0.9, 1.1)
tremolo_speed = np.random.uniform(0.1, 100)
tremolo_depth = np.random.randint(1, 101)
snr_range = np.random.randint(12, 100)

# Define the effect chain using f-strings
effects = [
    ["gain", "-n", f"{gain}"],
    ["chorus", *map(str, chorus_params)],
    ["overdrive", f"{drive}"],
    ["pitch", f"{pitch}"],
    ["reverb", *[str(param) for param in reverb_params]],
    ["speed", f"{speed}"],
    ["stretch", f"{stretch}"],
    ["tremolo", f"{tremolo_speed}", f"{tremolo_depth}"],
]

positive, _ = sox.apply_effects_tensor(waveform, sample_rate, effects)
positive = positive.mean(dim=0, keepdim=True)  # convert stereo to mono

positive_noisy = add_noise_with_snr(positive, snr_range=snr_range)

In [ ]:
Audio(positive_noisy, rate=sample_rate)

In [ ]:
#save the audio
#torchaudio.save('positive_noisy_blues10.wav', positive_noisy, sample_rate)

In [ ]:
def generate_negative(positive=positive_noisy, 
                      default_sample_rate: int = 44100,
                      min_chunk_duration_sec: float = 0.05,
                      max_chunk_duration_sec: float = 1.0):
    # Get positive length and duration
    positive_length = positive.shape[-1]
    positive_duration = positive_length / default_sample_rate

    # Determine the number of chunks based on minimum chunk duration
    n_chunks = int(positive_duration // min_chunk_duration_sec)

    # Calculate the minimum and maximum chunk lengths in samples
    min_chunk_length = int(min_chunk_duration_sec * default_sample_rate)
    max_chunk_length = int(max_chunk_duration_sec * default_sample_rate)

    # Generate random chunk lengths
    chunk_lengths = np.random.randint(min_chunk_length, max_chunk_length + 1, size=n_chunks - 1)
    chunk_lengths = np.append(chunk_lengths, positive_length - np.sum(chunk_lengths))

    # Split the positive clip into chunks
    chunks = [positive[..., start:start + length].clone().detach() for start, length in zip(np.cumsum(np.insert(chunk_lengths, 0, 0)), chunk_lengths)]

    # Shuffle the chunks
    np.random.shuffle(chunks)

    # Concatenate the shuffled chunks to create the negative example
    #TODO crossfade?
    negative = torch.cat(chunks, dim=-1)

    # Check if the positive and negative examples have the same length
    if positive.shape != negative.shape:
        raise ValueError(f"Input positive and output negative have different shapes: {positive.shape} vs {negative.shape}")

    return negative

In [ ]:
negative_from_positive = generate_negative(positive)
#TODO generate negative from anchor as well?

In [ ]:
 # Play the scrambled audio
Audio(negative_from_positive, rate=sample_rate)